# AMADEUS CHALLENGE

In [2]:
#name of files
!ls -l ../

total 7755000
drwxrwxr-x 4 dsc dsc       4096 may 28 07:48 amadeus_challenge
-rw-rw-r-- 1 dsc dsc 4244874509 abr  2  2018 bookings.csv
-rw-rw-r-- 1 dsc dsc 3696229366 abr  2  2018 searches.csv


In [51]:
#import libraries
import pandas as pd
import io
import requests

In [3]:
#Exploring

## Exercise 1: Count the number of lines in Python for each line

In [37]:
#Read files with only one column to reduce the size of the file
bookings_reader=pd.read_csv("../bookings.csv", delimiter="^", iterator=True, chunksize = 10000, usecols=[0])
searches_reader=pd.read_csv("../searches.csv", delimiter="^", iterator=True, chunksize = 10000, usecols=[0])


In [38]:
bookings_reader.get_chunk()

,act_date
0,2013-03-05 00:00:00
1,2013-03-26 00:00:00
2,2013-03-26 00:00:00
3,2013-03-26 00:00:00
4,2013-03-26 00:00:00
5,2013-03-20 00:00:00
6,2013-03-20 00:00:00
7,2013-03-25 00:00:00
8,2013-03-25 00:00:00
9,2013-03-25 00:00:00


In [5]:
def file_lines (file):
    length = 0
    for b_chunk in file:
        length += b_chunk.shape[0]
        print("*", end="")
    return length

bookings_len = file_lines(bookings_reader)
searches_len = file_lines(searches_reader)

print("Total lines of 'bookings.csv': {}".format(bookings_len))
print("Total lines of 'searches.csv': {}".format(searches_len))

*************

KeyboardInterrupt: 

In [ ]:
#pending.... Check for insosisnte date
#print(bookings_chunk["arr_port"].isnull().sum())

In [ ]:
#Clean file
#Bookings_first_chunk = 

## Exercise 2: Top 10 arrivals

In [30]:
#Read file including only "arr_port" and "pax" to calculate top 10 airports
chunks_booking=pd.read_csv("../bookings.csv", usecols=["arr_port","pax"],delimiter="^", iterator=True, chunksize = 10000 )#names=[arr_port,pax]

In [31]:
#Read all the chuncks aggregate data by airport ("arr_port") adding the bookings passangers ("pax")
pieces = [x.groupby('arr_port').sum() for x in chunks_booking]

In [129]:
#Concatenate all the pieces
agg = pd.concat(pieces).groupby(level=0).sum()

In [131]:
#Reset index, strip the names, and assign back the index
agg.reset_index(inplace=True)
agg["arr_port"]=agg["arr_port"].str.strip()
agg.set_index("arr_port", inplace=True)
#print top 10 airports per number of passangers
agg.iloc[:,0].sort_values(ascending=False).head(10)

arr_port
LHR    88809.0
MCO    70930.0
LAX    70530.0
LAS    69630.0
JFK    66270.0
CDG    64490.0
BKK    59460.0
MIA    58150.0
SFO    58000.0
DXB    55590.0
Name: pax, dtype: float64

In [139]:
#Importing airport names from GeoBases
url="https://raw.githubusercontent.com/opentraveldata/geobases/public/GeoBases/DataSources/Airports/GeoNames/airports_geonames_only_clean.csv"
columns=["iata_code","airport_name","country_name"]

iata_names = pd.read_csv(url, delimiter ="^",usecols=[0,1,4],header=None, names=columns)
iata_names["iata_code"]=iata_names["iata_code"].str.strip()
iata_names.set_index("iata_code", inplace=True)

In [145]:
#merge files to include names of country and airprt
aux=agg.join(iata_names, how="left")

#print top 10 airports per number of passangers including airport name and country
aux.sort_values(by="pax",ascending=False).head(10)

,pax,airport_name,country_name
arr_port,,,
LHR,88809.0,London Heathrow Airport,United Kingdom
MCO,70930.0,Orlando International Airport,United States
LAX,70530.0,Los Angeles International Airport,United States
LAS,69630.0,McCarran International Airport,United States
JFK,66270.0,John F Kennedy International Airport,United States
CDG,64490.0,Paris - Charles-de-Gaulle,France
BKK,59460.0,Suvarnabhumi,Thailand
MIA,58150.0,Miami International Airport,United States
SFO,58000.0,San Francisco International Airport,United States
